In [35]:
import bs4 as bs
import requests

import pandas as pd

import os

import time
import random

import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.sql import text

import duckdb


import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

In [36]:
# union of all season and team to one table for each table
servername = '****'
football_data = 'Football_Data'
trusted_conneciton = '?trusted_conneciton=yes'
driver = '&driver=ODBC+Driver+17+for+SQL+Server'

In [37]:
#create engine url to connect to sql server
engine = create_engine(f'mssql+pyodbc://{servername}/{football_data}{trusted_conneciton}{driver}')
engine

Engine(mssql+pyodbc://DESKTOP-5IAPFQC\MSSQLSERVER2019/Football_Data?driver=ODBC+Driver+17+for+SQL+Server&trusted_conneciton=yes)

In [38]:
con = engine.connect()

In [40]:
for position in position_li[1:2]:
    file = f'{position.upper()}_Player_Weekly_Stats'
    df = pd.read_csv(f'Weekly Stats/'+file, index_col=0)
    df['Receptions'] = df['Recptions']
    df.drop(columns=['Recptions'], inplace=True)
    df.to_sql(file, con=con, if_exists='replace')
    df.to_csv(f'Weekly Stats/'+file)

55

In [21]:
years = range(2010,2024)

In [22]:
weeks = range(1,18)

In [23]:
position_li = ['qb', 'rb', 'wr', 'te', 'k', 'dl', 'lb', 'db', 'dst']

In [8]:
def clean_df(df, year, week, position):
    try:
        df = df.droplevel(0, axis=1)

    except:
        pass

    df['Player'] = df['Player'].apply(lambda x: x.split(' ')[:2])
    df['Player'] = df['Player'].apply(lambda x: ' '.join(x))

    #Add Position Column
    df['Position'] = [position.upper()] * len(df)

    #add Year Column
    df['Year'] = [year] * len(df)

    df['Week'] = [week] * len(df)

    return df

In [26]:
def clean_dst_df(df, year, week, position):
    try:
        df = df.droplevel(0, axis=1)

    except:
        pass

    df['Player'] = df['Player'].apply(lambda x: x.split(' ')[-2])

    #Add Position Column
    df['Position'] = [position.upper()] * len(df)

    #add Year Column
    df['Year'] = [year] * len(df)

    df['Week'] = [week] * len(df)

    return df

In [10]:
def qb_col(df):
    #quarterback columns
    df.columns = ['Rank', 'Player', 'Pass Completions', 'Pass Attempts',
                  'Pass Completion/Attempts', 'Passing Yards', 'Passing Yards/Attempts', 'Passing Touchdowns', 
                  'Interception', 'Sacks', 'Rushing Attempts', 'Rushing Yards', 
                  'Rushing Touchdown', 'Fumbles', 'Games Played', 'Fantasy Football Points', 
                  'FTPS/Game', 'ROST', 'Position', 'Year', 'Week']
    for i in df.index:
        if df['Rushing Attempts'].iloc[i] == 0 and df['Rushing Yards'].iloc[i] > 0:
            df['Rushing Attempts'].iloc[i] = 1
    
    return df

In [11]:
def rb_col(df):
    #Running Back
    df.columns = ['Rank', 'Player', 'Rushing Attempts', 'Rushing Yards', 'Rushing Yards/Attempts',
                  'Longest Rushing', 'Rushing 20+', 'Rushing Touchdown',
                  'Receptions', 'TGT', 'Receiving Yards', 'Receiving Yards/Reception',
                  'Receiving Touchdowns', 'Fumbles', 'Games Played', 'Fantasy Football Points', 'FTPS/Game', 'ROST',
                  'Position', 'Year', 'Week']
    for i in df.index:
        if df['Rushing Attempts'].iloc[i] == 0 and df['Rushing Yards'].iloc[i] > 0:
            df['Rushing Attempts'].iloc[i] = 1

        if df['TGT'].iloc[i] == 0 and df['Receiving Yards'].iloc[i] > 0:
            df['TGT'].iloc[i] = 1

            
    return df

In [12]:
def wr_col(df):
    #wide receiver
    df.columns = ['Rank', 'Player', 'Receiving Receptions', 'Receiving Target', 'Receiving Yards',
                  'Receiving Yards/Reception', 'Longest Reception', 'Receiving 20+',
                  'Receiving Touchdowns', 'Rushing Attempts', 'Rushing Yards',
                  'Rushing Touchdown', 'Fumbles', 'Games Played', 'Fantasy Football Points', 'FTPS/Game', 'ROST',
                  'Position', 'Year', 'Week']
    for i in df.index:
        if df['Rushing Attempts'].iloc[i] == 0 and df['Rushing Yards'].iloc[i] > 0:
            df['Rushing Attempts'].iloc[i] = 1
            
    return df

In [13]:
def te_col(df):
    #tight end
    df.columns = ['Rank', 'Player', 'Receiving Receptions', 'Receiving Target', 'Receiving Yards',
                  'Receiving Yards/Reception', 'Longest Reception', 'Receiving 20+',
                  'Receiving Touchdowns', 'Rushing Attempts', 'Rushing Yards',
                  'Rushing Touchdown', 'Fumbles', 'Games Played', 'Fantasy Football Points',
                   'FTPS/Game', 'ROST', 'Position', 'Year', 'Week']

    for i in df.index:
        if df['Rushing Attempts'].iloc[i] == 0 and df['Rushing Yards'].iloc[i] > 0:
            df['Rushing Attempts'].iloc[i] = 1
            
    return df

In [14]:
def k_col(df):
    #kicking
    df.columns = ['Rank', 'Player', 'Field Goals', 'Field Attempts', 'Percentage',
                  'Longest Field', '1-19 Yards', '20-29 Yards', '30-39 Yards',
                  '40-49 Yards', '50+ Yards', 'Extra Points Made', 'Extra Points Attempted',
                  'Games Played', 'Fantasy Football Points',
                   'FTPS/Game', 'ROST', 'Position', 'Year', 'Week']
    return df

In [16]:
def dl_col(df):
    # defensive lineman
    df.columns = ['Rank', 'Player', 'Tackle', 'Assist', 
                  'Sack', 'Pass Defended', 'Interception', 'Forced Fumble', 
                  'Fumble Recovery', 'Defend Touchdown', 'Games Played', 'Fantasy Football Points', 
                  'FTPS/Game', 'ROST', 'Position', 'Year', 'Week']
    return df

In [17]:
def lb_col(df):
    #line backer
    df.columns = ['Rank', 'Player', 'Tackle', 'Assist', 'Sack', 'Pass Defended',
                  'Interception', 'Forced Fumble', 'Fumble Recovery', 'Defend Touchdown',
                  'Games Played', 'Fantasy Football Points', 'FTPS/Game', 'ROST', 'Position', 'Year', 'Week']    
    return df

In [18]:
def db_col(df):
    # defensive back
    df.columns = ['Rank', 'Player', 'Tackle', 'Assist', 'Sack', 'Pass Defended',
                  'Interception', 'Forced Fumble', 'Fumble Recovery', 'Defend Touchdown',
                  'Games Played', 'Fantasy Football Points', 'FTPS/Game', 'ROST', 'Position', 'Year', 'Week']
    return df

In [15]:
def def_st_col(df):

    df['Team'] = df['Player']
    # Defese and Special Teams
    df.columns = ['Rank', 'Player', 'Sack', 'Interceptions', 
                  'Fumble Recovery', 'Forced Fumble', 'Defense Touchdown', 'Safety', 
                  'Special Touchdown', 'Games Played', 'Fantasy Football Points', 'FTPS/Game', 
                  'ROST', 'Position', 'Year', 'Week', 'Team']
    return df

In [27]:
# webscrape positions
for year in years:    
    for week in weeks:
        for position in position_li[-1:]:

            print(year, week, position)
            
            url = f'https://www.fantasypros.com/nfl/stats/{position}.php?week={week}&range=week&year={year}'

            df = pd.read_html(url)[0]

            time.sleep(random.randrange(7,10))

            if position == 'dst':
                df = clean_dst_df(df, year, week, position)
            else:
                df = clean_df(df, year, week, position)
            
            if position == 'qb': 
                df = qb_col(df)
            elif position == 'rb':
                df = rb_col(df)
            elif position == 'wr':
                df = wr_col(df)
            elif position == 'te':
                df = te_col(df)
            elif position == 'k':
                df = k_col(df)
            elif position == 'dst':
                df = def_st_col(df)
            elif position == 'dl':
                df = dl_col(df)
            elif position == 'lb':
                df = lb_col(df)
            elif position == 'db':
                df = db_col(df)
            else:
                print('DNE', position, year)


            df.to_csv(f'NFL_Data_{year}/Weekly Stats/{position}_{week}_{year}')

2010 1 dst
2010 2 dst
2010 3 dst
2010 4 dst
2010 5 dst
2010 6 dst
2010 7 dst
2010 8 dst
2010 9 dst
2010 10 dst
2010 11 dst
2010 12 dst
2010 13 dst
2010 14 dst
2010 15 dst
2010 16 dst
2010 17 dst
2011 1 dst
2011 2 dst
2011 3 dst
2011 4 dst
2011 5 dst
2011 6 dst
2011 7 dst
2011 8 dst
2011 9 dst
2011 10 dst
2011 11 dst
2011 12 dst
2011 13 dst
2011 14 dst
2011 15 dst
2011 16 dst
2011 17 dst
2012 1 dst
2012 2 dst
2012 3 dst
2012 4 dst
2012 5 dst
2012 6 dst
2012 7 dst
2012 8 dst
2012 9 dst
2012 10 dst
2012 11 dst
2012 12 dst
2012 13 dst
2012 14 dst
2012 15 dst
2012 16 dst
2012 17 dst
2013 1 dst
2013 2 dst
2013 3 dst
2013 4 dst
2013 5 dst
2013 6 dst
2013 7 dst
2013 8 dst
2013 9 dst
2013 10 dst
2013 11 dst
2013 12 dst
2013 13 dst
2013 14 dst
2013 15 dst
2013 16 dst
2013 17 dst
2014 1 dst
2014 2 dst
2014 3 dst
2014 4 dst
2014 5 dst
2014 6 dst
2014 7 dst
2014 8 dst
2014 9 dst
2014 10 dst
2014 11 dst
2014 12 dst
2014 13 dst
2014 14 dst
2014 15 dst
2014 16 dst
2014 17 dst
2015 1 dst
2015 2 dst
201

In [22]:
df

,Rank,Player,Sack,Interceptions,Fumble Recovery,Forced Fumble,Defense Touchdown,Safety,Special Touchdown,Games Played,Fantasy Football Points,FTPS/Game,ROST,Position,Year,Week
0,1,Los Angeles,8,0,3,3,0,0,1,1,27.0,27.0,16.2%,DST,2023,9
1,2,Indianapolis Colts,4,3,0,1,2,0,0,1,26.0,26.0,24.7%,DST,2023,9
2,3,Cleveland Browns,7,2,1,1,0,0,0,1,23.0,23.0,94.2%,DST,2023,9
3,4,Las Vegas,8,2,0,1,0,0,0,1,19.0,19.0,22.2%,DST,2023,9
4,5,Baltimore Ravens,4,1,1,1,0,0,0,1,15.0,15.0,87.4%,DST,2023,9
5,6,New Orleans,2,3,2,2,0,0,0,1,13.0,13.0,63.8%,DST,2023,9
6,7,Green Bay,1,1,1,1,0,0,0,1,12.0,12.0,26.0%,DST,2023,9
7,8,Kansas City,3,0,1,2,1,0,0,1,12.0,12.0,77.0%,DST,2023,9
8,9,Los Angeles,4,0,2,2,0,0,0,1,9.0,9.0,21.1%,DST,2023,9
9,10,Atlanta Falcons,4,0,2,2,0,1,0,1,9.0,9.0,16.6%,DST,2023,9


In [28]:
# Concat each position by year and upload to sql MSSQLSERVER2019
for position in position_li[-1:]:
    
    position_df = pd.DataFrame()
    
    for year in years:

        file_li = os.listdir(f'NFL_Data_{year}/Weekly Stats/')

        for file in file_li:

            if position in file:
                df = pd.read_csv(f'NFL_Data_{year}/Weekly Stats/{file}', index_col=0)
                
                position_df = pd.concat([position_df, df], axis=0)


    position_df.to_sql(f'{position.upper()}_Player_Weekly_Stats', con=con, if_exists='replace')
    position_df.to_csv(f'Weekly Stats/{position.upper()}_Player_Weekly_Stats')

In [21]:
# get team for each player data from sql, join TEAM column, save in SQL and CSV
for position in position_li[:-1]:

    #join all years of that position
    query = f'''select {position}.*, Roster.Team
    from {position.upper()}_Player_Weekly_Stats {position}
    join Roster 
    on ({position}.Player = Roster.Player) 
    and ({position}.Year = Roster.Year)
    and ({position}.Position = Roster.Official_Position);
    '''

    # read sql
    position_df = pd.read_sql(query, con)
    position_df.drop(columns=['index'], inplace=True)

    position_df.to_csv(f'Weekly Stats/{position.upper()}_Player_Weekly_Stats')
    position_df.to_sql(f'{position.upper()}_Player_Weekly_Stats', con=con, if_exists='replace')

In [32]:
#join date from schedule and game results to position weekly stats
for position in position_li[-1:]:

    query = f'''select {position}.*, schedule_and_game_results.Date
    from {position.upper()}_Player_Weekly_Stats {position}
    join schedule_and_game_results 
    on ({position}.Year = schedule_and_game_results.Year)
    and ({position}.Week = schedule_and_game_results.Week)
    and ({position}.Team = schedule_and_game_results.Team);
    '''

    # read sql
    position_df = pd.read_sql(query, con)
    position_df.drop(columns=['index'], inplace=True)

    
    position_df.to_sql(f'{position.upper()}_Player_Weekly_Stats', con=con, if_exists='replace')
    position_df.to_csv(f'Weekly Stats/{position.upper()}_Player_Weekly_Stats')

In [ ]:
stop

In [ ]:
#different website for player data

In [ ]:
url = 'https://www.fantasypros.com/nfl/stats/qb.php?week=1&range=week&year=2010'

In [ ]:
df = pd.read_html(url)[0]

In [ ]:
df = df.droplevel(0, axis=1)
df

In [ ]:
df['Player'] = df['Player'].apply(lambda x: x.split(' ')[:2])
df['Player'] = df['Player'].apply(lambda x: ' '.join(x))

In [ ]:
url = 'https://www.nfl.com/stats/player-stats/'

In [ ]:
pd.read_html(url)

In [ ]:
a=[1,3,2]

In [ ]:
a.sort()

In [ ]:
a.unique()